In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow


In [ ]:
#Video = cv2.VideoCapture("/content/project1.mp4")
Video = cv2.VideoCapture("/content/test.mp4")
fps = Video.get(cv2.CAP_PROP_FPS)
totalNoFrames =Video.get(cv2.CAP_PROP_FRAME_COUNT);
durationInSeconds = float(totalNoFrames) / float(fps)

print("durationInSeconds: ",durationInSeconds,"s")
print("Total no of frames: ",totalNoFrames)

durationInSeconds:  8.008000000000001 s
Total no of frames:  240.0


In [ ]:
# Closest proximity check function
def find_if_close(cnt1,cnt2, d = 10):
    x, y, w, h = cv2.boundingRect(cnt1)
    coord1 = (x, y, x+w, y+h)
    centerCoord1 = np.array((coord1[0] + (coord1[2] / 2), coord1[1] + (coord1[3] / 2)))
    x, y, w, h = cv2.boundingRect(cnt2)
    coord2 = (x, y, x + w, y + h)
    centerCoord2 = np.array((coord2[0] + (coord2[2] / 2), coord2[1] + (coord2[3] / 2)))
    
    #print('frame = ',framenum)
    #print(centerCoord1,end = '\t')
    #print(centerCoord2)
    
    distance = np.linalg.norm(centerCoord1 - centerCoord2)
    #print("distance = " , distance)
    #print("abs_distance = " , abs(distance))
    if abs(distance) < d:
        return True
    else:
        return False

In [ ]:
# Creating a Background Substractor kernel
fgbg = cv2.createBackgroundSubtractorKNN(128, cv2.THRESH_BINARY, 1)
w_list = []
h_list = []
framenum = 0;

In [ ]:
from PIL import Image

In [ ]:
count = 0
hw_ratio = []
label = []
width_list = []
for i in range(50) : 
    print(-1)
while(framenum < totalNoFrames):
    framenum += 1
    # Obtain frame
    ret, frame = Video.read()
    # Convert the frame into Grey
    frame_grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Blur the the grey frame 
    frame_blur = cv2.GaussianBlur(frame_grey,(3,3),0) # (img , kernel , sigma(x) value ) , to remove noise or clear blur
    # Apply the Background Substraction mask
    fgmask = fgbg.apply(frame_blur)
    fgmask[fgmask == 127] = 0
    # Threshold the Background Substracted frame to remove grey values
    (threshold, frame_bw) = cv2.threshold(fgmask, 127, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    # Dilate the frame
    frame_bw = cv2.dilate(frame_bw, None, iterations=2)
    # Find the contours
    contours,hierarchy = cv2.findContours(frame_bw, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # cv2.RETR_EXTERNAL

    if(framenum > 50): ## CAN EDIT THE LIMIT
        # Develop a list of large contours
        contours_thresholded = []
        # Threshold the area for determining an contour is large enough or not
        threshold_area = 1
        v = 0

        for cnt in contours:
            area = cv2.contourArea(cnt)

            if area > threshold_area:
                contours_thresholded.append(cnt)
                v += 1

        # Sort the largest contours
        largest_contours = sorted(contours_thresholded, key=cv2.contourArea)[-5:]
        #print("largest contours : " , largest_contours)
        
        # Determine the biggest contour
        contour_sizes = [(cv2.contourArea(contour), contour) for contour in contours]
        biggest_contour = max(contour_sizes, key=lambda x: x[0])[1]
        # Determine the co-ordinates of the bounding box for the biggest contour
        (xx, yy, ww, hh) = cv2.boundingRect(biggest_contour)
        
        #print("height/width = " , hh/ww);
        #print(framenum)
       

        # Determine the number of largest contours
        LENGTH = len(largest_contours)

        # Set a array that will store the nearest bounding boxes
        rect = np.zeros((LENGTH * LENGTH, 4))

        # Index for tracking the closest bounding boxes
        f = 0
        if LENGTH == 1:
            (x1, y1, w1, h1) = cv2.boundingRect(largest_contours[0])
            rect[f] = (x1, y1, x1 + w1, y1 + h1)
            f += 1
            #print(-1)

        else:
            # Compare pairwise bounding boxes for the proximity and combine them
            for i, cnt1 in enumerate(largest_contours):
                #print("cnt1 : ", len(cnt1))
                #print("i = ",i)
                x = i
                if i != LENGTH - 1:
                    for j, cnt2 in enumerate(largest_contours[i + 1:]):
                       # print("cnt2 :", len(cnt2))
                        x = x + 1

                        # Call closest proximity check function
                        dist = find_if_close(cnt1, cnt2, 500)
                        if dist == True: 
                            #print("Inside if")
                            (x1, y1, w1, h1) = cv2.boundingRect(cnt1)
                            #print('x1,y1,w1,h1 = ' , (x1,y1,w1,h1))
                            (x2, y2, w2, h2) = cv2.boundingRect(cnt2)
                            #print('x1,y1,w1,h1 = ' , (x1,y1,w1,h1))
                            # Find the extreme values of the close bboxes
                            xl = min(x1, x2)
                            yl = min(y1, y2)
                            a = np.array((xl, yl))
                            xr = max(x1 + w1, x2 + w2)
                            yr = max(y1 + h1, y2 + h2)
                            b = np.array((xr, yr))
                            diag = np.linalg.norm(a - b)
                            # Store the diagonal points
                            rect[f] = (xl, yl, xr, yr)
                            f += 1

        # Find the extreme ends of the closest bounding boxes
        X = np.array(rect[(rect[:, 0] != 0), 0])
        Y = np.array(rect[(rect[:, 1] != 0), 1])
        X1 = int(min(X))
        Y1 = int(min(Y))
        X2 = int(max(rect[:, 2]))
        Y2 = int(max(rect[:, 3]))

        # list of difference between the two legs for all the frames
        w_list.append(ww)
        c = cv2.rectangle(fgmask, (X1, Y1),(X2, Y2), (255,0,0), 1)
        # Leg 1
        cv2.circle(frame, (xx + ww, Y2), 2, (0, 255, 0), -1)
        # Leg 2
        cv2.circle(frame, (xx, Y2), 2, (255, 0, 0), -1)

        # Alert when the person is falling
        height = Y2-Y1
        h_list_length = len(h_list)
        if h_list_length > 0:
            height_thresh = (2/3)*(sum(h_list)/len(h_list))
            if height < height_thresh:
                font = cv2.FONT_HERSHEY_SIMPLEX 
                cv2.putText(frame,  'FALLING!',  (50, 50),  font, 1,  (0, 0, 255),  2,  cv2.LINE_4) 
                
                t = Video.get(cv2.CAP_PROP_POS_MSEC)
                #print(t/1000) # timestamp in sec
                print("1") # denote fall
            else:
                font = cv2.FONT_HERSHEY_SIMPLEX 
                cv2.putText(frame,  'NO FALL !',  (50, 50),  font, 1,  (255,0,0),  2,  cv2.LINE_4) 
                print(-1)          

        # List of height of the bounding box for all frames
        h_list.append(height)
        #H_list.append(hh)
        #count = count +1
        # The feed showing the bounding box
        #cv2.imshow('Bounding box', c)
        #cv2_imshow(c)
        # The feed showing the feet points
        cv2_imshow(frame)

        # Plot of difference between both the feet
        #plt.plot(w_list)
        ## CAN CHANGE THE PLOT CONDITIONS
        #plt.xticks(np.arange(0,100,2))
        #plt.axes(ylim=[0, 400], xlim=[0, 100])
        plt.pause(0.1)
        
        #print(h_list == H_list)
        

Video.release()
cv2.destroyAllWindows()